In [681]:
import json
from statistics import mean
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import matplotlib.dates

In [682]:
input_csv_file = 'out.csv'

In [683]:
df = pd.read_csv(input_csv_file)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [684]:
df

,timestamp,OpenAI_gpt-3.5-turbo-0613 success,OpenAI_gpt-3.5-turbo-0613 output_characters,OpenAI_gpt-3.5-turbo-0613 first_token_time_seconds,OpenAI_gpt-3.5-turbo-0613 first_sentence_time_seconds,OpenAI_gpt-3.5-turbo-0613 total_request_time_seconds,OpenAI_gpt-3.5-turbo-0613 chunks_time_seconds,OpenAI_gpt-3.5-turbo-0125 success,OpenAI_gpt-3.5-turbo-0125 output_characters,OpenAI_gpt-3.5-turbo-0125 first_token_time_seconds,...,Azure_gpt-4-1106-preview first_token_time_seconds,Azure_gpt-4-1106-preview first_sentence_time_seconds,Azure_gpt-4-1106-preview total_request_time_seconds,Azure_gpt-4-1106-preview chunks_time_seconds,Azure_gpt-3.5-turbo-0613 success,Azure_gpt-3.5-turbo-0613 output_characters,Azure_gpt-3.5-turbo-0613 first_token_time_seconds,Azure_gpt-3.5-turbo-0613 first_sentence_time_seconds,Azure_gpt-3.5-turbo-0613 total_request_time_seconds,Azure_gpt-3.5-turbo-0613 chunks_time_seconds
0,2024-04-05 08:54:04.144802,True,493,0.367718,0.513484,1.226745,"[0.026351213455200195, 0.00017762184143066406,...",True,572,0.984307,...,3.557844,3.562629,10.723284,"[0.00032973289489746094, 3.1598007678985596, 0...",True,493,0.580326,0.583024,1.335795,"[0.0010137557983398438, 0.3710341453552246, 0...."
1,2024-04-05 08:54:45.221750,True,493,0.443277,0.580211,1.284911,"[0.022695064544677734, 0.0001766681671142578, ...",True,572,0.804435,...,15.254819,15.259835,23.070081,"[0.00032639503479003906, 9.591192722320557, 0....",True,493,0.546610,0.549194,1.332726,"[0.000244140625, 0.30342841148376465, 0.000169..."
2,2024-04-05 08:54:58.080303,True,493,0.954071,1.173532,1.847193,"[0.030196666717529297, 0.0001633167266845703, ...",True,512,0.463680,...,1.727435,1.732477,5.137654,"[0.0004088878631591797, 1.3246839046478271, 0....",True,493,0.448620,0.451217,1.228586,"[0.0003352165222167969, 0.28115129470825195, 0..."
3,2024-04-05 08:55:59.346834,True,493,0.513437,0.649116,1.337490,"[0.00017023086547851562, 0.0002424716949462890...",True,586,0.897318,...,0.000000,0.000000,0.000000,[],True,493,0.466268,0.468670,1.057621,"[0.00030493736267089844, 0.2541530132293701, 0..."
4,2024-04-05 08:56:07.438318,True,493,0.489830,0.629069,1.313012,"[0.02596902847290039, 0.0001633167266845703, 0...",True,487,0.918361,...,1.809049,1.814747,8.065334,"[0.00021910667419433594, 1.4208316802978516, 0...",True,493,0.440338,0.443052,0.983467,"[0.0003142356872558594, 0.24433279037475586, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2024-04-05 12:51:47.152863,True,493,0.339886,0.516188,1.454693,"[0.035730600357055664, 0.00017189979553222656,...",True,512,0.301437,...,1.790291,1.795321,7.390486,"[0.0002875328063964844, 1.2774693965911865, 0....",True,493,0.567715,0.570399,1.271996,"[0.0003218650817871094, 0.3690938949584961, 0...."
475,2024-04-05 12:52:19.194390,True,493,0.717799,0.751579,1.787066,"[0.0001785755157470703, 0.0002503395080566406,...",True,591,0.620947,...,5.131990,5.138554,9.427131,"[0.00032401084899902344, 4.641822099685669, 0....",True,493,0.735197,0.737764,1.584778,"[0.00023889541625976562, 0.3881232738494873, 0..."
476,2024-04-05 12:52:47.265670,True,493,0.344106,0.501259,1.302861,"[0.029307126998901367, 0.00016927719116210938,...",True,512,0.437146,...,2.289865,2.294828,7.499413,"[0.00030112266540527344, 1.837409496307373, 0....",True,493,0.354743,0.357390,0.938646,"[0.00033211708068847656, 0.17971301078796387, ..."
477,2024-04-05 12:53:18.484529,True,493,0.570938,0.778046,1.598528,"[0.03412485122680664, 0.00016570091247558594, ...",True,589,0.379572,...,3.322076,3.333693,8.716894,"[0.0002963542938232422, 1.9504432678222656, 0....",True,493,0.456194,0.458905,1.243726,"[0.0003256797790527344, 0.29770994186401367, 0..."


In [685]:
plt.figure(figsize=(16, 9), dpi=300)

plt.subplot(2, 2, 1)

gpt_4_models = ["gpt-4-1106-preview", "gpt-4-0125-preview"]
gpt_3_models = ["gpt-3.5-turbo-0613", "gpt-3.5-turbo-0125"]

models = gpt_4_models + gpt_3_models
providers = ["OpenAI", "Azure"]

for model in models:
    for provider in providers:
        column_name = f'{provider}_{model} total_request_time_seconds'
        if column_name not in df.columns:
            continue
        
        sns.lineplot(x='timestamp', y=column_name, data=df, label=f'{provider} {model}', linestyle="--" if provider == "Azure" else "-")

plt.xlabel('Time')
plt.ylabel('Seconds')
plt.title('Total Request time')
plt.xticks(rotation=45)
plt.legend()

plt.subplot(2, 2, 2)
for model in models:
    for provider in providers:
        column_name = f'{provider}_{model} first_token_time_seconds'
        if column_name not in df.columns:
            continue
        
        sns.lineplot(x='timestamp', y=column_name, data=df, label=f'{provider} {model}', linestyle="--" if provider == "Azure" else "-")

plt.xlabel('Time')
plt.ylabel('Seconds')
plt.title('Time to first token')
plt.xticks(rotation=45)
plt.legend()

plt.subplot(2, 2, 3)
for model in models:
    for provider in providers:
        column_name = f'{provider}_{model} first_sentence_time_seconds'
        if column_name not in df.columns:
            continue
        
        sns.lineplot(x='timestamp', y=column_name, data=df, label=f'{provider} {model}', linestyle="--" if provider == "Azure" else "-")

plt.xlabel('Time')
plt.ylabel('Seconds')
plt.title('Time to first sentence')
plt.xticks(rotation=45)
plt.legend()

plt.subplot(2, 2, 4)
for model in models:
    for provider in providers:
        column_name = f'{provider}_{model} output_characters'
        if column_name not in df.columns:
            continue
        
        sns.lineplot(x='timestamp', y=column_name, data=df, label=f'{provider} {model}', linestyle="--" if provider == "Azure" else "-")

plt.xlabel('Time')
plt.ylabel('Seconds')
plt.title('Number of generated characters')
plt.xticks(rotation=45)
plt.legend()

plt.tight_layout()
plt.savefig('misc/line_charts.png')
plt.close()

In [686]:
def plot_models(_models, col_name, col_title, file_name):

    plt.figure(figsize=(16, 9), dpi=300)
    
    added_labels = set()
    
    for model in _models:
        for provider in providers:
            column_name = f'{provider}_{model} {col_name}'
            success_column_name = f'{provider}_{model} success'
            
            if column_name not in df.columns:
                continue
                
            filtered_df = df[df[success_column_name] == True]    
            sns.lineplot(x='timestamp', y=column_name, data=filtered_df, label=f'{provider} {model}', linestyle="--" if provider == "Azure" else "-")
            
            failure_dates = df['timestamp'][df[success_column_name] == False].unique()
            for date in failure_dates:
                failure_label = f'{provider} {model} API Timeout'
                
                if failure_label not in added_labels:
                    plt.axvline(x=date, color='r', linestyle="dashdot" if provider == "Azure" else "dotted", linewidth=1, label=failure_label)
                    added_labels.add(failure_label)
                else:
                    plt.axvline(x=date, color='r', linestyle="dashdot" if provider == "Azure" else "dotted", linewidth=1, label=None)
                
    ax = plt.gca()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M UTC'))
    
            
    plt.xlabel('Time')
    plt.ylabel('Seconds')
    plt.title(col_title)
    plt.xticks(rotation=45)
    plt.savefig(file_name)
    plt.legend()
    plt.close()


In [687]:
plot_models(gpt_4_models, "total_request_time_seconds", "GPT-4 Total Request time", "misc/gpt_4_total_request_time_seconds.png")
plot_models(gpt_3_models, "total_request_time_seconds", "GPT-3 Total Request time", "misc/gpt_3_total_request_time_seconds.png")

In [688]:
plot_models(gpt_4_models, "first_token_time_seconds", "GPT-4 Time to first token", "misc/gpt_4_first_token_time_seconds.png")
plot_models(gpt_3_models, "first_token_time_seconds", "GPT-3 Time to first token", "misc/gpt_3_first_token_time_seconds.png")

In [689]:
plot_models(gpt_4_models, "first_sentence_time_seconds", "GPT-4 Time to first finished sentence", "misc/gpt_4_first_sentence_time_seconds.png")
plot_models(gpt_3_models, "first_sentence_time_seconds", "GPT-3 Time to first finished sentence", "misc/gpt_3_first_sentence_time_seconds.png")

In [690]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_models_failure_bar_chart(_models, col_title, file_name):
    plt.figure(figsize=(16, 9), dpi=300)
    
    failure_counts = [] 
    labels = []
    
    for model in _models:
        for provider in providers:
            success_column_name = f'{provider}_{model} success'
            
            if success_column_name not in df.columns:
                continue
            
            failure_df = df[df[success_column_name] == False]
            
            failure_counts.append(failure_df.shape[0])
            labels.append(f'{provider} {model}')
    

    sns.barplot(x=labels, y=failure_counts)
    
    plt.ylabel('Number of Failures')
    plt.title(col_title)
    plt.savefig(file_name)
    
    plt.close()


In [691]:
plot_models_failure_bar_chart(gpt_4_models, "GPT-4 Number of failures to finish the request in time", "misc/gpt_4_num_failures.png")
plot_models_failure_bar_chart(gpt_3_models, "GPT-3 Number of failures to finish the request in time", "misc/gpt_3_num_failures.png") 

In [692]:
def plot_models_time_per_chunk(_models, col_title, file_name):
    plt.figure(figsize=(16, 9), dpi=300)
    
    boxplot_data = []
    labels = []
    
    for model in _models:
        for provider in providers:
            column_name = f'{provider}_{model} chunks_time_seconds'
            success_column_name = f'{provider}_{model} success'
            
            if column_name not in df.columns:
                continue

            filtered_df = df[df[success_column_name] == True]

            for index, row in filtered_df.iterrows():
                
                for chunk_time in json.loads(row[column_name]):
                    boxplot_data.append(chunk_time)
                    labels.append(f'{provider} {model}')

    plot_df = pd.DataFrame({'Seconds': boxplot_data, 'Model': labels})
    
    sns.boxplot(x='Model', y='Seconds', data=plot_df)
    
    plt.title(col_title)
    plt.savefig(file_name)
    plt.close()

In [693]:
plot_models_time_per_chunk(gpt_4_models, "GPT-4 Time between each streaming chunk", "misc/gpt_4_chunks_timing.png")
plot_models_time_per_chunk(gpt_3_models, "GPT-3 Time between each streaming chunk", "misc/gpt_3_chunks_timing.png")

In [694]:
def plot_models_time_per_chunk_line(_models, col_title, file_name):
    plt.figure(figsize=(16, 9), dpi=300)
    
    for model in _models:
        for provider in providers:
            column_name = f'{provider}_{model} chunks_time_seconds'
            success_column_name = f'{provider}_{model} success'
            
            if column_name not in df.columns:
                continue

            filtered_df = df[df[success_column_name] == True]
            
            chunk_times = []
            chunk_nums = []

            for index, row in filtered_df.iterrows():
                
                for i, chunk_time in enumerate(json.loads(row[column_name])):
                    chunk_times.append(chunk_time)
                    chunk_nums.append(i)

            plot_df = pd.DataFrame({'Seconds': chunk_times, "Chunk Number": chunk_nums})
            sns.lineplot(x='Chunk Number', y='Seconds', data=plot_df, label=f'{provider} {model}')


    plt.title(col_title)

    plt.savefig(file_name)
    plt.close()

In [695]:
plot_models_time_per_chunk_line(gpt_4_models, "GPT-4 Time between each streaming chunk", "misc/gpt_4_chunks_timing_line.png")
plot_models_time_per_chunk_line(gpt_3_models, "GPT-3 Time between each streaming chunk", "misc/gpt_3_chunks_timing_line.png")

In [696]:


def plot_models_time_per_chunk_mean(_models, col_title, file_name):
    plt.figure(figsize=(16, 9), dpi=300)
    
    model_provider_values = defaultdict(list)
    
    for model in _models:
        for provider in providers:
            column_name = f'{provider}_{model} chunks_time_seconds'
            success_column_name = f'{provider}_{model} success'
            
            if column_name not in df.columns:
                continue

            filtered_df = df[df[success_column_name] == True]
            
            for index, row in filtered_df.iterrows():
                for chunk_time in json.loads(row[column_name]):
                    model_provider_values[f'{provider} {model}'].append(chunk_time)

    model_means = []
    model_labels = []
    
    for key, value in model_provider_values.items():
        model_means.append(sum(value) / len(value))
        model_labels.append(key)

    sns.barplot(x=model_labels, y=model_means)

    plt.title(col_title)
    plt.savefig(file_name)
    plt.close()

In [697]:
plot_models_time_per_chunk_mean(gpt_4_models, "GPT-4 Mean of time between chunks", "misc/gpt_4_chunks_timing_means.png")
plot_models_time_per_chunk_mean(gpt_3_models, "GPT-3 Mean of time between chunks", "misc/gpt_3_chunks_timing_means.png")

In [698]:
from statistics import median


def plot_models_time_per_chunk_median(_models, col_title, file_name):
    plt.figure(figsize=(16, 9), dpi=300)
    
    model_provider_values = defaultdict(list)
    
    for model in _models:
        for provider in providers:
            column_name = f'{provider}_{model} chunks_time_seconds'
            success_column_name = f'{provider}_{model} success'
            
            if column_name not in df.columns:
                continue

            filtered_df = df[df[success_column_name] == True]
            
            for index, row in filtered_df.iterrows():
                for chunk_time in json.loads(row[column_name]):
                    model_provider_values[f'{provider} {model}'].append(chunk_time)

    model_means = []
    model_labels = []
    
    for key, value in model_provider_values.items():
        model_means.append(median(value))
        model_labels.append(key)

    sns.barplot(x=model_labels, y=model_means)

    plt.title(col_title)
    plt.savefig(file_name)
    plt.close()


In [699]:
plot_models_time_per_chunk_median(gpt_3_models, "GPT-4 Median of time between chunks", "misc/gpt_3_chunks_timing_median.png")
plot_models_time_per_chunk_median(gpt_4_models, "GPT-3 Median of time between chunks", "misc/gpt_4_chunks_timing_median.png")

In [700]:
def plot_models_bar_chart(_models, col_name, col_title, file_name, use_median=True):

    plt.figure(figsize=(16, 9), dpi=300)
    
    labels = []
    values = []
    
    for model in _models:
        for provider in providers:
            column_name = f'{provider}_{model} {col_name}'
            success_column_name = f'{provider}_{model} success'
            
            if column_name not in df.columns:
                continue
                
            filtered_df = df[df[success_column_name] == True]
            
            # print(filtered_df[column_name].values)
            
            labels.append(f'{provider} {model}')
            
            if use_median:
                values.append(median(filtered_df[column_name].values))
            else:
                values.append(mean(filtered_df[column_name].values))

    sns.barplot(x=labels, y=values)

    
    plt.ylabel('Seconds')
    plt.title(col_title)
    plt.savefig(file_name)
    plt.close()


In [701]:
plot_models_bar_chart(gpt_3_models, "first_token_time_seconds", "GPT-3 Median time for first token", "misc/gpt_3_first_token_time_seconds_bar_median.png")
plot_models_bar_chart(gpt_4_models, "first_token_time_seconds", "GPT-4 Median time for first token", "misc/gpt_4_first_token_time_seconds_bar_median.png")

In [702]:
plot_models_bar_chart(gpt_3_models, "first_sentence_time_seconds", "GPT-3 Median time for first finished sentence", "misc/gpt_3_first_sentence_time_seconds_bar_median.png")
plot_models_bar_chart(gpt_4_models, "first_sentence_time_seconds", "GPT-4 Median time for first finished sentence", "misc/gpt_4_first_sentence_time_seconds_bar_median.png")

In [703]:
plot_models_bar_chart(gpt_3_models, "total_request_time_seconds", "GPT-3 Median time for complete request", "misc/gpt_3_total_request_time_seconds_bar_median.png")
plot_models_bar_chart(gpt_4_models, "total_request_time_seconds", "GPT-4 Median time for complete request", "misc/gpt_4_total_request_time_seconds_bar_median.png")

In [704]:
plot_models_bar_chart(gpt_3_models, "first_token_time_seconds", "GPT-3 Mean time for first token", "misc/gpt_3_first_token_time_seconds_bar_mean.png", use_median=False)
plot_models_bar_chart(gpt_4_models, "first_token_time_seconds", "GPT-4 Mean time for first token", "misc/gpt_4_first_token_time_seconds_bar_mean.png", use_median=False)

In [705]:
plot_models_bar_chart(gpt_3_models, "first_sentence_time_seconds", "GPT-3 Mean time for first finished sentence", "misc/gpt_3_first_sentence_time_seconds_bar_mean.png", use_median=False)
plot_models_bar_chart(gpt_4_models, "first_sentence_time_seconds", "GPT-4 Mean time for first finished sentence", "misc/gpt_4_first_sentence_time_seconds_bar_mean.png", use_median=False)

In [706]:
plot_models_bar_chart(gpt_3_models, "total_request_time_seconds", "GPT-3 Mean time for complete request", "misc/gpt_3_total_request_time_seconds_bar_mean.png", use_median=False)
plot_models_bar_chart(gpt_4_models, "total_request_time_seconds", "GPT-4 Mean time for complete request", "misc/gpt_4_total_request_time_seconds_bar_mean.png", use_median=False)

In [707]:
def plot_stacked_model(_provider_left, _model_left, _provider_right, _model_right, file_name):
    success_column_name = f'{_provider_left}_{_model_left} success'
    filtered_df = df[df[success_column_name] == True]
    
    total_first_token_column_name_left = f'{_provider_left}_{_model_left} first_token_time_seconds'
    total_first_sentence_column_name_left = f'{_provider_left}_{_model_left} first_sentence_time_seconds'
    total_time_column_name_left = f'{_provider_left}_{_model_left} total_request_time_seconds'
    
    total_first_token_column_name_right = f'{_provider_right}_{_model_right} first_token_time_seconds'
    total_first_sentence_column_name_right = f'{_provider_right}_{_model_right} first_sentence_time_seconds'
    total_time_column_name_right = f'{_provider_right}_{_model_right} total_request_time_seconds'
    
    new_df_left = pd.DataFrame()
    new_df_left['timestamp'] = filtered_df['timestamp']
    new_df_left['first_token_column_name'] = filtered_df[total_first_token_column_name_left]
    new_df_left['first_sentence_time_seconds'] = filtered_df[total_first_sentence_column_name_left] - filtered_df[total_first_token_column_name_left]
    new_df_left['remaining_time_seconds'] = filtered_df[total_time_column_name_left] - filtered_df[total_first_sentence_column_name_left] - filtered_df[total_first_token_column_name_left]
    
    new_df_right = pd.DataFrame()
    new_df_right['timestamp'] = filtered_df['timestamp']
    new_df_right['first_token_column_name'] = filtered_df[total_first_token_column_name_right]
    new_df_right['first_sentence_time_seconds'] = filtered_df[total_first_sentence_column_name_right] - filtered_df[total_first_token_column_name_right]
    new_df_right['remaining_time_seconds'] = filtered_df[total_time_column_name_right] - filtered_df[total_first_sentence_column_name_right] - filtered_df[total_first_token_column_name_right]
    

    fig, axs = plt.subplots(1, 2, figsize=(20, 6), sharey='row',  dpi=300) 

    # Left model plot
    axs[0].stackplot(
        new_df_left['timestamp'], 
        new_df_left['first_token_column_name'], 
        new_df_left['first_sentence_time_seconds'], 
        new_df_left['remaining_time_seconds'], 
        labels=['First token', 'First finished sentence', 'Remaining time']
    )
    axs[0].xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M UTC'))
    axs[0].legend(loc='upper left')
    axs[0].set_title(f'Stacked Time Components for {_provider_left} {_model_left}')
    axs[0].set_ylabel('Seconds')
    axs[0].tick_params(axis='x', rotation=45)

    axs[1].stackplot(
        new_df_right['timestamp'], 
        new_df_right['first_token_column_name'], 
        new_df_right['first_sentence_time_seconds'], 
        new_df_right['remaining_time_seconds'], 
        labels=['First token', 'First finished sentence', 'Remaining Time']
    )
    axs[1].xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M UTC'))
    axs[1].legend(loc='upper left')
    axs[1].set_title(f'Stacked Time Components for {_provider_right} {_model_right}')
    axs[1].set_ylabel('Seconds')
    axs[1].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.savefig(file_name)
    plt.close()
    plt.show()
    

plot_stacked_model("OpenAI", "gpt-4-1106-preview", "Azure", "gpt-4-1106-preview", "misc/compare_azure_openai_gpt-4-1106-preview.png")
plot_stacked_model("OpenAI", "gpt-3.5-turbo-0613", "Azure", "gpt-3.5-turbo-0613", "misc/compare_azure_openai_gpt-3.5-turbo-0125.png")